In [17]:
from google.cloud import bigquery

In [20]:
# Import libraries

from google.cloud import bigquery
import pandas as pd
from pandas_gbq import to_gbq
import os
#print('Libraries imported successfully')

In [34]:
# Initialize BigQuery client
client = bigquery.Client()


In [21]:
#!pip3 install --upgrade pip
# !pip install google-cloud-bigquery
# !pip install pandas-gbq -U
# !pip install db-dtypes
# !pip install packaging --upgrade


# Set the environment variable for Google Cloud credentials
# Place the path in which the .json file is located.

# Example (if .json is located in the same directory with the notebook)
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "at-arch-416714-6f9900ec7.json"

# -- YOUR CODE GOES BELOW THIS LINE
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/armela/Desktop/neat-calculus-453319-f5-08fadb59b569.json" # Edit path
# -- YOUR CODE GOES ABOVE THIS LINE


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1
  Using cached google_api_core-2.24.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_cloud_core-2.4.3-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached google_crc32c-1.7.1-cp312-cp312-macosx_12_0_arm64.whl.metadata (2.3 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
Using cached google_api_core-2.24.2-py3-none-any.whl (160 kB)
Using cached cachetools-5.5.2-py3-none-any.whl (10 kB)
Using cached google_cloud_core-2.4.3-py2.py3-none-any.whl (29 kB)
Us

In [35]:
## Define your project and dataset

project_id = 'neat-calculus-453319-f5'
dataset_id = 'reporting_db'
table_id = 'rep_revenue_per_customer_and_period'

In [41]:
from google.cloud import bigquery

# Initialize BigQuery client
client = bigquery.Client()

# Write the query

create_or_replace_query = """
CREATE OR REPLACE TABLE `neat-calculus-453319-f5.reporting_db.rep_revenue_per_customer_and_period` AS

WITH valid_payments AS (
  SELECT
    p.payment_amount,
    p.payment_date,
    r.customer_id
  FROM `neat-calculus-453319-f5.staging_db.stg_payment` p
  JOIN `neat-calculus-453319-f5.staging_db.stg_rental` r
    ON p.rental_id = r.rental_id
  JOIN `neat-calculus-453319-f5.staging_db.stg_inventory` i
    ON r.inventory_id = i.inventory_id
  JOIN `neat-calculus-453319-f5.staging_db.stg_film` f
    ON i.film_id = f.film_id
  WHERE f.film_title != 'GOODFELLAS SALUTE'
)

SELECT 
  customer_id,
  'Day' AS reporting_period,
  DATE_TRUNC(payment_date, DAY) AS reporting_date,
  SUM(payment_amount) AS total_revenue
FROM valid_payments
WHERE payment_date >= '2015-01-01'
GROUP BY customer_id, DATE_TRUNC(payment_date, DAY)

UNION ALL

SELECT 
  customer_id,
  'Month' AS reporting_period,
  DATE_TRUNC(payment_date, MONTH) AS reporting_date,
  SUM(payment_amount) AS total_revenue
FROM valid_payments
WHERE payment_date >= '2015-01-01'
GROUP BY customer_id, DATE_TRUNC(payment_date, MONTH)

UNION ALL

SELECT 
  customer_id,
  'Year' AS reporting_period,
  DATE_TRUNC(payment_date, YEAR) AS reporting_date,
  SUM(payment_amount) AS total_revenue
FROM valid_payments
WHERE payment_date >= '2015-01-01'
GROUP BY customer_id, DATE_TRUNC(payment_date, YEAR)
"""

# Execute the query

query_job = client.query(create_or_replace_query)  
query_job.result()  




In [58]:
query = """
SELECT * 
FROM `neat-calculus-453319-f5.reporting_db.rep_revenue_per_customer_and_period`
LIMIT 10
"""

query_job = client.query(query)
results = query_job.result()

for row in results:
    print(row)


TypeError: 'str' object is not callable